## Joint MCMC Sampling
This notebook attemps to sample from the joint posterior of planck and LSST. The emulators are used to calculate the likelihood. For parameters in planck but not LSST the joint prior is extended to be uninformative in the new parameters. Since the experiments are independent the joint likelihood is simply the sum/product.

In [17]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from getdist import plots, MCSamples
import getdist
from multiprocessing import Pool
from getdist import plots, MCSamples, WeightedSamples

import sys
import time
import os

sys.path.append(os.path.join(os.path.dirname("__file__"), '../'))
# LSST
from LSST_emulation.cocoa_emu import *
from LSST_emulation.cocoa_emu.emulator import NNEmulator, GPEmulator
from LSST_emulation.cocoa_emu.data_model import LSST_3x2

# cosmopower
sys.path.append(os.path.join(os.path.dirname("__file__"), '../'))
import cosmopower.likelihoods.tf_planck2018_lite as cppl
import cosmopower as cp

import emcee
import time

# Now normalizing flow
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tf.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import Callback

from numpy import linalg
import scipy

%matplotlib inline

In [18]:
### Planck MCMC
ipynb_path = os.path.dirname(os.path.realpath("__file__"))


tt_emu_model = cp.cosmopower_NN(restore=True,
                                restore_filename=os.path.join(ipynb_path, '../cosmopower_emcee/cosmopower/trained_models/CP_paper/CMB/cmb_TT_NN')
                               )

te_emu_model = cp.cosmopower_PCAplusNN(restore=True,
                                restore_filename=os.path.join(ipynb_path, '../cosmopower_emcee/cosmopower/trained_models/CP_paper/CMB/cmb_TE_PCAplusNN')
                               )

ee_emu_model = cp.cosmopower_NN(restore=True,
                                restore_filename=os.path.join(ipynb_path, '../cosmopower_emcee/cosmopower/trained_models/CP_paper/CMB/cmb_EE_NN')
                               )

# path to the tf_planck2018_lite likelihood
#tf_planck2018_lite_path = './../cosmopower/likelihoods/tf_planck2018_lite/'

# parameters of the analysis, and their priors
parameters_and_priors = {'omega_b':      [0.001, 0.04, 'uniform'],
                         'omega_cdm':    [0.005, 0.99,  'uniform'],
                         'h':            [0.2,   1.0,   'uniform'],
                         'tau_reio':     [0.01,  0.8,   'uniform'],
                         'n_s':          [0.9,   1.1,   'uniform'],
                         'ln10^{10}A_s': [1.61,  3.91,  'uniform'],
                         'A_planck':     [1.0,   0.01,  'gaussian'],
                          }

# instantiation
#tf_planck = cp.tf_planck2018_lite(parameters=parameters_and_priors, 
#                                  tf_planck2018_lite_path=tf_planck2018_lite_path,
#                                  tt_emu_model=tt_emu_model,
#                                  te_emu_model=te_emu_model,
#                                  ee_emu_model=ee_emu_model
#                                  )

# initial points
FIDUCIAL = np.reshape(np.array([ 0.022242,  0.11977,  0.673,  0.055,  0.9658,  3.0753,  1.0080]).astype('float32'), (1, 7))
EPSILON  = np.reshape(np.array([ 1E-4,      1E-4,     1E-2,   1E-3,   1E-3,    1E-3,    1E-4]).astype('float32'), (1,7))

# instantiation
tf_planck = cppl(parameters=parameters_and_priors, 
                                  tf_planck2018_lite_path='/home/grads/data/evan/cosmopower_emcee/cosmopower/likelihoods/tf_planck2018_lite',
                                  tt_emu_model=tt_emu_model,
                                  te_emu_model=te_emu_model,
                                  ee_emu_model=ee_emu_model
                                  )

In [19]:
### LSST MCMC Functions
def add_bias(bias_theta, datavector):
    for i in range(5):
        factor = (bias_theta[i] / bias_fid[i])**bias_mask[i]
        datavector = factor * datavector
    return datavector

def add_shear_calib(m, datavector):
    for i in range(5):
        factor = (1 + m[i])**shear_calib_mask[i]
        datavector = factor * datavector
    return datavector

def hard_prior(theta, params_prior):
    """
    A function to impose a flat prior on a set of parameters.
    :theta: The set of parameter values
    :params_prior: The minimum and the maximum value of the parameters on which this prior is imposed
    """
    is_lower_than_min = bool(np.sum(theta < params_prior[:,0]))
    is_higher_than_max = bool(np.sum(theta > params_prior[:,1]))
    if is_lower_than_min or is_higher_than_max:
        return -np.inf
    else:
        return 0.
    
cosmo_prior_lim = np.array([[1.61, 3.91],# planck [1.61,  3.91,  'uniform'],
                       [0.87, 1.07], # planck [0.9,   1.1,   'uniform']
                       [55, 91],
                       [0.01, 0.04], # [0.001, 0.04, 'uniform'],
                       [0.001, 0.99]]) # [0.005, 0.99,  'uniform'],

# parameters of the analysis, and their priors

ia_prior_lim = np.array([[-5., 5.],
                       [-5., 5.]])

bias_prior_lim = np.array([[0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.],
                       [0.8, 3.]])

baryon_prior_lim = np.array([[-3., 12.],
                             [-2.5, 2.5]])

baryon_prior_lim = 3. * baryon_prior_lim 

dz_source_std   = 0.002 * np.ones(5)
dz_lens_std     = 0.005 * np.ones(5)
shear_calib_std = 0.005 * np.ones(5)
    
def lnprior(theta):
    cosmo_theta = theta[:5]
    ns          = cosmo_theta[1]

    ns_prior    = 0.
    
    dz_source   = theta[5:10]
    ia_theta    = theta[10:12]
    dz_lens     = theta[12:17]
    bias        = theta[17:22]
    shear_calib = theta[22:27]
    baryon_q    = theta[27:29]
    
    cosmo_prior = hard_prior(cosmo_theta, cosmo_prior_lim) + ns_prior
    ia_prior    = hard_prior(ia_theta, ia_prior_lim)
    bias_prior  = hard_prior(bias, bias_prior_lim)
    baryon_prior = hard_prior(baryon_q, baryon_prior_lim)
    
    dz_source_lnprior   = -0.5 * np.sum((dz_source / dz_source_std)**2)
    dz_lens_lnprior     = -0.5 * np.sum((dz_lens / dz_lens_std)**2)
    shear_calib_lnprior = -0.5 * np.sum((shear_calib / shear_calib_std)**2)
    
    return cosmo_prior + ia_prior + dz_source_lnprior + dz_lens_lnprior + \
            shear_calib_lnprior + bias_prior + baryon_prior
    
def ln_lkl(theta):
    model_datavector = get_data_vector_emu(theta)
    delta_dv = (model_datavector - data_model.dv_obs)[data_model.mask_3x2]
    return -0.5 * delta_dv @ data_model.masked_inv_cov @ delta_dv

def get_data_vector_emu(theta):
    """
    Function to get the emulated data vector (including the effect of galaxy bias, baryons, etc.)
    """
    cosmo_ia_dz_theta = theta[:17]
    bias        = theta[17:22]
    shear_calib = theta[22:27]
    baryon_q    = theta[27:29]
    datavector = data_model.compute_datavector(cosmo_ia_dz_theta)
    datavector = np.array(datavector)
    datavector = add_bias(bias, datavector)
    datavector = add_shear_calib(shear_calib, datavector)
    return datavector

In [20]:
### Planck MCMC functions
#
# from LSST emulator for index reference
# names = ['logA', 'ns', 'H0', 'omegab', 'omegac']

As a reminder, the parameters are as follows:
- LSST: labels =  ['logA', 
        'ns', 
        'H0', 
        'omega b', 
        'omega c'] 
        in [:,:5]
- Planck: names = [r"\omega_{\mathrm{b}}", 
         r"\omega_{\mathrm{cdm}}", 
         r"h", 
         r"\tau",
         r"n_s", 
         r"ln 10^{10} A_s", 
         r"A_{\mathrm{Planck}}"]
         
They have in common the densities $\Omega_b$ and $\Omega_c$, the spectral index $n_s$, the amplitude $A_s$. This means we must extend the LSST the priors in 3 extra dimensions, one for $\tau$, one for $h$, and one for the planck calibration amplitude $A_{\mathrm{Planck}}$.

In [21]:
# open the emulator
# Get the LSST covariance and fid data
path = '/home/grads/data/evan/LSST_emulation/data/lsst_y1/'
lsst_cov = np.loadtxt(path+'cov_lsst_y1')
fid_cos = np.loadtxt(path+'lsst_y1_data_fid',dtype=np.float32)[:,1]

lsst_y1_cov = np.zeros((1560, 1560))
for line in lsst_cov:
    i = int(line[0])
    j = int(line[1])

    cov_g_block  = line[-2]
    cov_ng_block = line[-1]

    cov_ij = cov_g_block + cov_ng_block

    lsst_y1_cov[i,j] = cov_ij
    lsst_y1_cov[j,i] = cov_ij
    
fid = torch.Tensor(fid_cos)
cov = torch.Tensor(lsst_y1_cov)

configfile = path+'../../configs/nn_emu.yaml'
config = Config(configfile)

config_args     = config.config_args
config_args_io  = config_args['io']
config_args_data = config_args['data']

savedir = path+'../../output/nn_emu/'

N_DIM         = 17
data_model    = LSST_3x2(N_DIM, config_args_io, config_args_data)
data_model.emu_type = 'nn'
OUTPUT_DIM = 1560

emu = NNEmulator(N_DIM, OUTPUT_DIM, data_model.dv_fid, data_model.dv_std)    
emu.load(path+'../../model/nn_emu/model')
# ======================================================

data_model.emu = emu

bias_fid         = data_model.bias_fid
bias_mask        = data_model.bias_mask
shear_calib_mask = data_model.shear_calib_mask

N_MCMC        = 5000
N_WALKERS     = 120
NDIM_SAMPLING = 32

theta0    = np.array([3.0675, 0.97, 69.0, 0.0228528, 0.1199772, 
                      0., 0., 0., 0., 0.,
                      0.5, 0.,
                      0., 0., 0., 0., 0.,
                      1.24, 1.36, 1.47, 1.60, 1.76,
                      0., 0., 0., 0., 0.,
                      0., 0.,1.05,0.05,1.0])

theta_std = np.array([0.01, 0.001, 0.1, 0.001, 0.002, 
                      0.002, 0.002, 0.002, 0.002, 0.002, 
                      0.1, 0.1,
                      0.005, 0.005, 0.005, 0.005, 0.005, 
                      0.03, 0.03, 0.03, 0.03, 0.03,
                      0.005, 0.005, 0.005, 0.005, 0.005, 
                      0.1, 0.1,0.01,0.01,0.01]) 

# Starting position of the emcee chain
pos0 = theta0[np.newaxis] + 3. * theta_std[np.newaxis] * np.random.normal(size=(N_WALKERS, NDIM_SAMPLING))

ModuleNotFoundError: No module named 'cocoa_emu'

In [ ]:
def planck_ln_lkl(theta):
    theta_p = np.zeros(6)
    theta_p[0] = theta[3] # omega b
    theta_p[1] = theta[4] # omega c
    theta_p[2] = theta[29] # h
    theta_p[3] = theta[30] # tau
    theta_p[4] = theta[0] # A_s
    theta_p[5] = theta[31] # A_planck
    p=tf_planck.posterior(theta_p.astype(np.float32)).numpy()
    return p    

def lnprob(theta):
    return lnprior(theta) + ln_lkl(theta) + planck_ln_lkl(theta)

In [ ]:
start_time = time.time()
#with Pool(10) as pool:
emu_sampler = emcee.EnsembleSampler(N_WALKERS, NDIM_SAMPLING+6, lnprob)#, pool=pool)
emu_sampler.run_mcmc(pos0, N_MCMC, progress=True)